In [2]:
import spotipy
import spotipy.oauth2 as oauth2
import pandas as pd
import time
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import gensim

In [3]:
from spotipy.oauth2 import SpotifyClientCredentials

client_id='38e94de386924d73b4098a6a28d45070'
client_secret='27899e6f31894a6aa3c572268f8658e5'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
spot = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



In [4]:
df = pd.read_csv('missing_songs.csv', header=None, names=['ind', 'song_id'])

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
df.shape

(2040661, 2)

In [7]:
song_ids = df.song_id

In [8]:
len(song_ids)

2040661

In [9]:
#missing feat
def get_track_artist(song_id):
    result =  spot.track(song_id)
    artist = result['artists'][0]['name']
    track = result['name']
    popularity = result['popularity']
    x = {'song_id': [song_id], 'artist': [artist], 'track': [track], 'popularity': [popularity]}
    df = pd.DataFrame(data=x)
    return df

In [10]:
# 1850000, 1950000

In [23]:
song_frames = []
fails = []

for i in tqdm(range(1900000,len(song_ids))):
    try:
        df = get_track_artist(song_ids[i])
        song_frames.append(df)
    except:
        fails.append(song_ids[i])

  5%|▌         | 7356/140661 [13:09<3:32:29, 10.46it/s] 

retrying ...1secs


 50%|█████     | 70524/140661 [2:09:18<1:51:37, 10.47it/s] 

retrying ...1secs


 54%|█████▍    | 76163/140661 [2:18:59<1:47:07, 10.03it/s]

...

 82%|████████▏ | 115076/140661 [3:38:29<43:04,  9.90it/s]  

retrying ...1secs


 82%|████████▏ | 115265/140661 [3:38:49<42:14, 10.02it/s]  

retrying ...1secs


100%|██████████| 140661/140661 [4:31:43<00:00,  8.32it/s]  


In [24]:
df = pd.concat(song_frames)

In [25]:
df.to_csv('scraped_song_ids.csv', mode='a', header=False)
pd.DataFrame(fails).to_csv('fails.csv')

In [26]:
x = pd.read_csv('scraped_song_ids.csv')

In [27]:
x.shape

(4009881, 5)

In [28]:
x.artist.shape

(4009881,)

In [29]:
x.artist.unique().shape

(419329,)

In [32]:
x.song_id.shape

(4009881,)

In [30]:
x.song_id.unique().shape

(2749996,)

In [33]:
old_playlishape= pd.read_csv('scraped_playlists.csv')
new_playlists = pd.read_csv('new_scraped_playlists.csv')

In [34]:
new_playlists.shape

(16254689, 6)

In [35]:
old_playlists.shape

(5319400, 6)

In [36]:
playlists = pd.concat([old_playlists,new_playlists])

In [38]:
#combine playlist dataframes check for dups and then make into master playlist frame to be merged
playlists.shape

(21574089, 6)

In [40]:
playlists.head()

,Unnamed: 0,song_id,playlist_id,playlist_name,user_id,followers
0,0,6HfVKTa0AA4k49FqXtxd3S,3nnKwI4n769FO2KPI3dfk1,stiles stilinski,_erin_morgan_,4.0
1,1,3pf6HWXtEalIWoZcPTFbc6,3nnKwI4n769FO2KPI3dfk1,stiles stilinski,_erin_morgan_,4.0
2,2,0ofHAoxe9vBkTCp2UQIavz,3nnKwI4n769FO2KPI3dfk1,stiles stilinski,_erin_morgan_,4.0
3,3,3dwvT66ZwtwlIZl3IISe6X,3nnKwI4n769FO2KPI3dfk1,stiles stilinski,_erin_morgan_,4.0
4,4,1t0UbGPCjGpLeH7KiLBuBv,3nnKwI4n769FO2KPI3dfk1,stiles stilinski,_erin_morgan_,4.0


In [42]:
playlists.drop(columns=['Unnamed: 0'],inplace=True)

In [43]:
playlists.shape[0] - playlists.drop_duplicates(subset=['song_id','playlist_id','playlist_name']).shape[0]

9966902

In [44]:
playlists.isnull().sum()

song_id          41490
playlist_id          0
playlist_name      437
user_id            283
followers         2897
dtype: int64

In [45]:
playlists.dropna(subset=['song_id'],inplace=True)

In [46]:
playlists.isnull().sum()

song_id             0
playlist_id         0
playlist_name     437
user_id           283
followers        2889
dtype: int64

In [47]:
playlists.shape[0] - playlists.drop_duplicates(subset=['song_id','playlist_id','playlist_name']).shape[0]

9930529

In [48]:
playlists.to_csv('playlists_final.csv')